In [6]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

stocks = pd.read_csv('sp_500_stocks.csv')

from secrets import IEX_CLOUD_API_TOKEN

stock_name = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{stock_name}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

stock_price = data['latestPrice']
market_cap = data['marketCap']

df_columns = ['Ticker', 'Stock Price', 'Market Capitalization', '# of Shares to Buy']
final_dataframe = pd.DataFrame(columns = df_columns)
final_dataframe

final_dataframe.append(
    pd.Series(
    [
        stock_name,
        stock_price,
        market_cap,
        'N/A'
    ],
    index = df_columns
    ),
    ignore_index=True
)



def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
name_groups = list(chunks(stocks['Ticker'], 100))
name_strings = []
for i in range(0, len(name_groups)):
    name_strings.append(','.join(name_groups[i]))
final_dataframe = pd.DataFrame(columns = df_columns)
for name_string in name_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={name_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for name in name_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            name,
            data[name]['quote']['latestPrice'],
            data[name]['quote']['marketCap'],
            'N/A'
        ],
        index = df_columns
        ),
        ignore_index = True   
        )
        
        
portfolioSize = input('Enter the value of portfolio: ')
try:
    val = float(portfolioSize)
except ValueError:
    print("That's not a number bruh. \nTry again:")
    portfolioSize = input('Enter the value of portfolio: ')
        
        
positionSize = val / len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i, '# of Shares to Buy'] = math.floor(positionSize/final_dataframe.loc[i, 'Stock Price'])

    
    
print(final_dataframe)

#writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
#final_dataframe.to_excel(writer, 'Recommended Trades', index = False)
        
        
        
#backgroundColor = '#0a0a23'
#fontColor = '#ffffff'

#stringFormat = writer.book.add_format(
#{
    #'font_color': fontColor,
    #'bg_color': backgroundColor,
    #'border': 1
#}
#)

#dollarFormat = writer.book.add_format(
#{
    #'num_format' : '$0.00',
    #'font_color': fontColor,
    #'bg_color': backgroundColor,
    #'border': 1
#}
#)

#integerFormat = writer.book.add_format(
#{
    #'num_format': '0',
    #'font_color': fontColor,
    #'bg_color': backgroundColor,
    #'border': 1
#}
#)
        
#writer.sheets['Recommended Trades'].write('A1', 'Ticker', stringFormat)
#writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollarFormat)
#writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollarFormat)
#writer.sheets['Recommended Trades'].write('D1', '# of Shares to Buy', integerFormat)
        

#columnFormats = {
    #'A': ['Ticker', stringFormat],
    #'B': ['Stock Price', dollarFormat],
    #'C': ['Market Capitalization', dollarFormat],
    #'D': ['# of Shares to Buy', integerFormat]
#}

#for column in columnFormats.keys():
    #writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, columnFormats[column][1])
    #writer.sheets['Recommended Trades'].write(f'{column}1', columnFormats[column][0], columnFormats[column][1])
        

#writer.save()

    
    
    
    

Enter the value of portfolio: 1000000
    Ticker  Stock Price Market Capitalization # of Shares to Buy
0        A       129.60           39600012531                 15
1      AAL        18.04           10553550684                109
2      AAP       160.24           10759784279                 12
3     AAPL       139.43         2318977307232                 14
4     ABBV       104.51          192374492070                 18
..     ...          ...                   ...                ...
500    YUM       109.08           32122635010                 18
501    ZBH       167.69           34764231106                 11
502   ZBRA       488.51           26813746086                  4
503   ZION        51.45            8427326337                 38
504    ZTS       172.69           79880668724                 11

[505 rows x 4 columns]
